# AutoGluon-Tabular による顧客離反分析 (Churn Analysis)

---

---

## コンテンツ

1. [背景](#1.背景)
1. [セットアップ](#2.セットアップ)
1. [データ](#3.データ)
1. [学習](#4.学習)
1. [推論と評価](#5.推論と評価)
---

## 1.背景

どのようなビジネスであっても、顧客を失うことは大きな損害です。もし、満足していない顧客を早期に見つけることができれば、そのような顧客をキープするためのインセンティブを提供できる可能性があるでしょう。このノートブックでは、満足していない顧客を自動で認識するために機械学習 (Machine Learning, ML) を利用する方法を説明します。このような顧客の離反分析は Customer Churn Prediction と呼ばれています。機械学習モデルは完璧な予測を行えないので、このノートブックでは予測のエラーが生じたときの相対的なコストを考慮して、機械学習を利用したときの成果を金額で評価します。

ここでは、私達にとってなじみのある離反分析、携帯電話会社からの離反を取り上げます。携帯電話会社が、ある顧客が離反しそうと察知したら、その顧客にタイムリーにインセンティブを与えます。つまり、電話をアップグレードしたり、新しい機能を使えるようになったりして、引き続き携帯電話会社を使おうと思うかもしれません。インセンティブは、顧客が離反して再度獲得するまでにかかるコストよりもずっと小さいことが多いです。

このサンプルノートブックでは、[AutoGluon](https://auto.gluon.ai/stable/tutorials/tabular_prediction/index.html) という AutoML のフレームワークを使って顧客離反予測モデルを学習します。AutoGluon は複数のモデルのアンサンブルを含めてモデルを学習します。学習データを分析して自動的に特徴量を生成したり前処理したりします。

Amazon SageMaker のビルトインアルゴリズムの XGBoost を使った方法は [こちら](https://github.com/aws-samples/amazon-sagemaker-examples-jp/tree/master/xgboost_customer_churn) をご参照ください。

---

## 2.セットアップ

まず、AutoGluon を使用するために必要なライブラリをインストールします。

In [ ]:
!pip install autogluon
!pip install bokeh

次に、このノートブックインスタンスに付与されている IAM role を `get_execution_role()` から取得しましょう。後ほど、SageMaker の学習やホスティングを行いますが、そこで IAM role が必要になります。そこで、ノートブックインスタンスの IAM role を、学習やホスティングでも利用します。
通常、role を取得するためにはAWS SDKを利用した数行のコードを書く必要があります。ここでは `get_execution_role()` のみで role を取得可能です。SageMaker Python SDK は、データサイエンティストが機械学習以外のコードを簡潔に済ませるために、このような関数を提供しています。


In [ ]:
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

以降で利用するライブラリをここで読み込んでおきます。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from IPython.display import HTML

print('Current SageMaker Python SDK Version ={0}'.format(sagemaker.__version__))

---
## 3.データ

携帯電話会社は、どの顧客が最終的に離反したか、または、サービスを使い続けたかの履歴データをもっています。この履歴データに対して学習を行うことで、携帯電話会社の顧客離反を予想するモデルを構築します。モデルの学習が終わった後、任意の顧客のデータ (モデルの学習で利用したものと同じ情報を利用します）をモデルに入力すると、モデルはその顧客が離反しそうかどうかを予測します。もちろん、モデルは誤って予測することも考えられるので、将来を予測することはやはり難しいですが、そのような誤りに対応する方法も紹介します。

ここで利用するデータセットは一般的に利用可能で、書籍 [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) の中で Daniel T. Larose が言及しているものです。そのデータセットは、著者によって University of California Irvine Repository of Machine Learning Datasets に提供されています。ここでは、そのデーセットをダウンロードして読み込んでみます。

Jupyter notebook では、冒頭に `!` を入力することで、シェルコマンドを実行することができます。AWS CLIを用いてS3からデータをダウンロードします。

In [ ]:
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt ./

カレントディレクトリ下にダウンロードされた `churn.txt` を `pandas` を利用して読み込んでみます。 `pandas` は、表形式のデータを読み込んで、様々な加工ができるライブラリです。例えば、以下を実行すると表形式でのデータ表示が可能です。

In [ ]:
churn = pd.read_csv('./churn.txt')
pd.set_option('display.max_columns', 500)
churn

データをみると 5,000 行のデータしかなく、現在の機械学習の状況から見ると、やや小さいデータセットです。各データのレコードは、ある米国の携帯電話会社の顧客のプロフィールを説明する21の属性からなります。その属性というのは、

- `State`: 顧客が居住している米国州で、2文字の省略形で記載されます (OHとかNJのように)
- `Account Length`: アカウントが利用可能になってからの経過日数
- `Area Code`: 顧客の電話番号に対応する3桁のエリアコード
- `Phone`: 残りの7桁の電話番号
- `Int’l Plan`: 国際電話のプランに加入しているかどうか (yes/no)
- `VMail Plan`: Voice mail の機能を利用しているかどうか (yes/no)
- `VMail Message`: 1ヶ月の Voice mail のメッセージの平均長
- `Day Mins`: 1日に通話した時間(分)の総和
- `Day Calls`: 1日に通話した回数の総和
- `Day Charge`: 日中の通話にかかった料金
- `Eve Mins, Eve Calls, Eve Charge`: 夜間通話にかかった料金
- `Night Mins`, `Night Calls`, `Night Charge`: 深夜通話にかかった料金
- `Intl Mins`, `Intl Calls`, `Intl Charge`: 国際通話にかかった料金
- `CustServ Calls`: カスタマーサービスに電話をかけた回数
- `Churn?`: そのサービスから離反したかどうか (true/false)

最後の属性 `Churn?` は目的変数として知られ、MLのモデルで予測する属性になります。目的変数は2値 (binary) なので、ここで作成するモデルは2値の予測を行います。これは2値分類といわれます。

それではデータを詳しく見てみます。

まずはカテゴリデータごとにデータの頻度をみてみます。カテゴリデータは、`State`, `Area code`, `Phone`, `Int’l Plan`, `VMail Plan`, `Churn?`で、カテゴリを表す文字列や数値がデータとして与えられているものです。`pandas`ではある程度自動で、カテゴリデータを認識し、`object`というタイプでデータを保存します。以下では、`object` 形式のデータをとりだして、カテゴリごとの頻度を表示します。

また `describe()`を利用すると各属性の統計量を一度に見ることができます。

In [ ]:
# Frequency tables for each categorical feature
for column in churn.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=churn[column], columns='% observations', normalize='columns'))

# Histograms for each numeric features
display(churn.describe())
%matplotlib inline
hist = churn.hist(bins=30, sharey=True, figsize=(10, 10))

In [ ]:
chrun_org = churn.copy()

データを見てみると以下のことに気づくと思います。

- `State` の各頻度はだいたい一様に分布しています。
- `Phone` はすべて同じ数値になっていて手がかりになりそうにありません。この電話番号の最初の3桁はなにか意味がありそうですが、その割当に意味がないのであれば、使うのは止めるべきでしょう
- 数値的な特徴量は都合の良い形で分布しており、多くは釣り鐘のようなガウス分布をしています。ただ、`VMail Message`は例外です。
- `Area code` は数値データとみなされているようなので、非数値に変換しましょう

さて、実際に`Phone`の列を削除して、`Area code`を非数値に変換します。

In [ ]:
churn = churn.drop('Phone', axis=1)
churn['Area Code'] = churn['Area Code'].astype(object)

それでは次に各属性の値を、目的変数の True か False か、にわけて見てみます。

In [ ]:
for column in churn.select_dtypes(include=['object']).columns:
    if column != 'Churn?':
        display(pd.crosstab(index=churn[column], columns=churn['Churn?'], normalize='columns'))

for column in churn.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = churn[[column, 'Churn?']].hist(by='Churn?', bins=30)
    plt.show()

データ分析の結果から、離反する顧客について、以下のような傾向が考えられます。

- 地理的にもほぼ一様に分散している
- 国際通話を利用している
- VoiceMailを利用していない
- 通話時間で見ると長い通話時間と短い通話時間の人に分かれる
- カスタマーサービスへの通話が多い (多くの問題を経験した顧客ほど離反するというのは理解できる)

加えて、離反する顧客に関しては、`Day Mins` と `Day Charge` で似たような分布を示しています。しかし、話せば話すほど、通常課金されるので、驚くことではないです。もう少し深く調べてみましょう。`corr()` を利用すると相関係数を求めることができます。

In [ ]:
display(churn.corr())
pd.plotting.scatter_matrix(churn, figsize=(12, 12))
plt.show()

いくつかの特徴は互いに100%の相関をもっています。このような特徴があるとき、機械学習のアルゴリズムによっては全くうまくいかないことがあり、そうでなくても結果が偏ったりしてしまうことがあります。これらの相関の強いペアは削除しましょう。Day Mins に対する Day Charge、Night Mins に対する Night Charge、Intl Mins に対する Intl Charge を削除します。

In [ ]:
churn = churn.drop(['Day Charge', 'Eve Charge', 'Night Charge', 'Intl Charge'], axis=1)

ここまででデータセットの前処理は完了です。これから利用する機械学習のアルゴリズムを決めましょう。前述したように、数値の大小 (中間のような数値ではなく)で離反を予測するような変数を用意すると良さそうです。線形回帰のようなアルゴリズムでこれを行う場合は、複数の項（もしくはそれらをまとめた項）を属性として用意する必要があります。

そのかわりに、これを勾配ブースティング木 (Gradient Boosted Tree)を利用しましょう。Amazon SageMaker は、マネージドで、分散学習が設定済みで、リアルタイム推論のためのホスティングも可能な XGBoost コンテナを用意しています。XGBoost は、特徴感の非線形な関係を考慮した勾配ブースティング木を利用しており、特徴間の複雑な関連性を扱うことができます。

Amazon SageMaker の XGBoostは、csv または LibSVM 形式のデータを学習することができます。ここでは csv を利用します。csv は以下のようなデータである必要があります。

- 1列目が予測対象のデータ
- ヘッダ行はなし

まずはじめに、カテゴリ変数を数値データに変換する必要があります。`get_dummies()` を利用すると数値データへの変換が可能です。

そして、`Churn?_True`をターゲット変数として使用するため、同等の情報量を持つ`Churn?_False.`は削除します。



In [ ]:
model_data = pd.get_dummies(churn).drop(columns=['Churn?_False.'])

ここでデータを学習用、テスト用データにわけましょう。これによって overfitting (学習用データには精度が良いが、実際に利用すると制度が悪い、といった状況) を回避しやすくなり、未知のテストデータに対する精度を確認することができます。

In [ ]:
train_data2, test_data2 = np.split(model_data.sample(frac=1, random_state=1729), [int(0.9 * len(model_data))])
train_data2.to_csv('train2.csv', header=False, index=False)

In [ ]:
train_data2.head()

## 4. 学習

それでは、 AutoGluon を使ってモデルを学習し、性能を確認します。まずは前処理をしていない生データを使ってモデルを学習してみます。AutoGluon はモデルの学習の際に学習データを自動的に学習用と検証用に分けてくれるので検証用データは作らず、データを学習用データとモデルの性能評価用のテストデータに分割します。

In [ ]:
train_data, test_data = np.split(chrun_org.sample(frac=1, random_state=1729), [int(0.9 * len(model_data))])
train_data.to_csv('train.csv', header=False, index=False)

In [ ]:
from autogluon.tabular import TabularPredictor

# ターゲット変数の列名を指定
COLUMN_NAME = 'Churn?'

predictor = TabularPredictor(label=COLUMN_NAME, path='raw_data').fit(train_data=train_data)

In [ ]:
results = predictor.fit_summary()

In [ ]:
HTML(filename='raw_data/SummaryOfModels.html')

各特徴量の重要度を確認します。importance の値が大きいほど正しい予測をするのに重要だと言えます。AutoGluon-Tabular は、各特徴量の値を列ごとにランダムにシャッフルして予測にどの程度影響があったかを測定することで、特徴量の重要度を推定しています。importance が負の場合は、その特徴量が予測にとって有害な可能性があることを示します。

- importance: 推定されたその特徴量の重要度
- stddev: 特徴量の重要度の標準偏差。NaNの場合は、分散を計算するのに十分な num_shuffle_sets が設定されていない。
- p_value: 帰無仮説を importance = 0、（片側）対立仮説を importance > 0 としたときの t検定の P値。P値が 0.01の場合、その特徴が役に立たないか有害である可能性が1%で、その特徴が有用である可能性が99%であることを示します。
- n: 重要度を推定するために行うシャッフルの回数
- p99_high: 特徴量の重要度の 99%信頼区間の上限
- p99_low: 特徴量の重要度の 99%信頼区間の下端

In [ ]:
predictor.feature_importance(test_data)

続いて、手動でデータの前処理をしたデータを使ってモデルの学習をしてみます。

In [ ]:
# ターゲット変数の列名を指定
COLUMN_NAME_MOD = 'Churn?_True.'

predictor_mod = TabularPredictor(label=COLUMN_NAME_MOD, path='mod_data').fit(train_data=train_data2)

In [ ]:
results = predictor_mod.fit_summary()

In [ ]:
HTML(filename='mod_data/SummaryOfModels.html')

In [ ]:
predictor_mod.feature_importance(test_data2)

---
## 5.推論と評価

学習が終わったら、テストデータを使ってモデルを評価しましょう。まずは、データの前処理をしていないデータを使って学習したモデルを評価します。テストデータを使って、学習された各モデルの性能を確認します。

In [ ]:
leaderboard = predictor.leaderboard(test_data)
leaderboard

In [ ]:
test_data_nolab = test_data.drop(columns=[COLUMN_NAME]) 
y_test = test_data[COLUMN_NAME] 
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Probability を出力したい場合は、`predict_proba` を使用します。

In [ ]:
perf_proba = predictor.predict_proba(test_data_nolab)
perf_proba

続いて、データの前処理をして学習したモデルの性能を確認します。

In [ ]:
leaderboard = predictor_mod.leaderboard(test_data2)
leaderboard

In [ ]:
test_data_nolab2 = test_data2.drop(columns=[COLUMN_NAME_MOD]) 
y_test2 = test_data2[COLUMN_NAME_MOD] 
y_pred2 = predictor_mod.predict(test_data_nolab2)
print("Predictions:  \n", y_pred2)
perf2 = predictor_mod.evaluate_predictions(y_true=y_test2, y_pred=y_pred2, auxiliary_metrics=True)
perf_proba2 = predictor_mod.predict_proba(test_data_nolab2)

 ### Accuracy の比較
 モデルの accuracy を比較してみます。データの特性などにも依存しますが、前処理なしで学習した方が高い性能のモデルを作れることもあります。ドメイン知識がない場合は、AutoGluon による自動的な前処理に任せるのも有効ですし、ドメイン知識がある場合は手作業で前処理する方が有効なこともあります。

In [ ]:
print('Accuracy without data preprocessing: ', perf['accuracy'])
print('Accuracy with data preprocessing: ',  perf2['accuracy'])

### 混同行列の比較
機械学習の性能を比較評価する方法はいくつかありますが、単純に、予測値と実際の値を比較しましょう。今回は、顧客が離反する `1` と離反しない `0` を予測しますので、この混同行列を作成します。

まずは前処理なしで学習したモデルの混同行列です。

In [ ]:
predictions_flag = pd.Series([int(s) for s in y_pred.replace('True.', '1').replace('False.', '0')])
test_pread_flag =  pd.Series([int(s) for s in test_data.iloc[:, -1].replace('True.', '1').replace('False.', '0')])

In [ ]:
pd.crosstab(index=test_pread_flag, columns=predictions_flag, rownames=['actual'], colnames=['predictions'])

次はは前処理ありで学習したモデルの混同行列です。

In [ ]:
pd.crosstab(index=test_data2.iloc[:,-1], columns=np.round(y_pred2), rownames=['actual'], colnames=['predictions'])

### しきい値の検討
実際のユースケースでは目的に応じて、Probability がいくつ以上だったら離反と見なすかをカスタマイズすること必要があります。

#### 推論エラーのコスト
2値分類の問題においては、しきい値に注意しなければならない状況に直面することが多いです。それ自体は問題ではありません。もし、出力の連続値が2クラスで完全に別れていれば、MLを使うことなく単純なルールで解くことができると考えられます。

重要なこととして、MLモデルを本番環境に導入する際、モデルが False Positives と False Negatives に誤って入れたときのコストがあげられます。しきい値の選択は4つの指標に影響を与えます。4つの指標に対して、ビジネス上の相対的なコストを考える必要があるでしょう。

コストの割当
携帯電話会社の離反の問題において、コストとはなんでしょうか？コストはビジネスでとるべきアクションに結びついています。いくつかの仮定をおいてみましょう。

まず、True Negatives のコストとして `$0` を割り当てます。満足しているお客様を正しく認識できていれば何も実施しません。

False Negatives が一番問題で、なぜなら、離反していく顧客を正しく予測できないからです。顧客を失えば、再獲得するまでに多くのコストを払う必要もあり、例えば逸失利益、広告コスト、管理コスト、販売管理コスト、電話の購入補助金などがあります。インターネットを簡単に検索してみると、そのようなコストは数百ドルとも言われ、ここでは `$500` としましょう。これが False Negatives に対するコストです。

最後に、離反していくと予測された顧客に `$100` のインセンティブを与えることを考えましょう。 携帯電話会社がそういったインセンティブを提供するなら、2回くらいは離反の前に考え直すかもしれません。これは True Positive と　False Negative のコストになります。False Positives の場合 (顧客は満足していて、モデルが誤って離反しそうと予測した場合)、 `$100` のインセンティブは捨てることになります。その `$100` を効率よく消費してしまうかもしれませんが、優良顧客へのロイヤリティを増やすという意味では悪くないかもしれません。

#### 最適なしきい値を探す
False Negatives が False Positives よりもコストが高いことは説明しました。そこで、顧客の数ではなく、コストを最小化するように、しきい値を最適化することを考えましょう。コストの関数は以下のようなものになります。

`$500 * FN(C) + $0 * TN(C) + $100 * FP(C) + $100 * TP(C)`

FN(C) は False Negative の割合で、しきい値Cの関数です。同様にTN, FP, TP も用意します。この関数の値が最小となるようなしきい値Cを探します。 最も単純な方法は、候補となるしきい値で何度もシミュレーションをすることです。以下では100個の値に対してループで計算を行います。

前処理なしのモデルの Probability を確認します。

In [ ]:
data = np.array(perf_proba['True.'].values.tolist())
plt.hist(data)
plt.show()

前処理ありのモデルの Probability を確認します。

In [ ]:
data2 = np.array(perf_proba2[1].values.tolist())
plt.hist(data2)
plt.show()

In [ ]:
cutoffs = np.arange(0.01, 1, 0.01)
costs = []
min_c = 1.0
cost_min = 1000000

for c in cutoffs:
    _predictions = pd.Categorical(np.where(data > c, 1, 0), categories=[0, 1])
    matrix_a = np.array([[0, 100], [500, 100]])
    matrix_b = pd.crosstab(index=test_pread_flag, columns=_predictions, dropna=False)
    cost = np.sum(np.sum(matrix_a * matrix_b))
    costs.append(cost)
    if cost < cost_min:
        min_c = c
        cost_min = cost

costs = np.array(costs)
plt.plot(cutoffs, costs)
plt.show()
print('Cost is minimized near a cutoff of:', cutoffs[np.argmin(costs)], 'for a cost of:', np.min(costs))

コスト関数に鑑みてしきい値を設定することで、実際に離反してしまった顧客を離反しないと予測してしまう False Negative の数が減りました。

In [ ]:
_predictions = pd.Categorical(np.where(data > min_c, 1, 0), categories=[0, 1])
pd.crosstab(index=test_pread_flag, columns=_predictions, dropna=False, rownames=['actual'], colnames=['predictions'])